# Extract the Lyrics of EXO's Songs From the Genius API

#### Install & Import the packages we need

In [1]:
#To install LyricsGenius (and get the most updated version from GitHub)
!pip install git+https://github.com/johnwmillr/LyricsGenius.git

  Cloning https://github.com/johnwmillr/LyricsGenius.git to /private/var/folders/05/9w4py2dx5n99dk_hh6cv2bf40000gn/T/pip-req-build-mjwijt8p
  Running command git clone --filter=blob:none --quiet https://github.com/johnwmillr/LyricsGenius.git /private/var/folders/05/9w4py2dx5n99dk_hh6cv2bf40000gn/T/pip-req-build-mjwijt8p
  Resolved https://github.com/johnwmillr/LyricsGenius.git to commit 486fd52b81325c49006b1f4ab3130cd890963f9f
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import lyricsgenius

from bs4 import BeautifulSoup
import re
import requests
from pathlib import Path

#### Connect to Genius

In [3]:
# You will need to go to Genius Developers to get your own client access toke
client_access_token = "Your-Client-Access-Token"

In [4]:
# Get Songs and Lyrics By a Specific Artist
LyricsGenius = lyricsgenius.Genius(client_access_token)

artist = LyricsGenius.search_artist("EXO", max_songs=2)

Searching for songs by EXO...

Song 1: "Ko Ko Bop"
Song 2: "전야 (前夜) (The Eve)"

Reached user-specified song limit (2).
Done. Found 2 songs.


In [5]:
artist.songs

for song in artist.songs:
    print(song.lyrics)

[엑소 "Ko Ko Bop" 가사]

[Intro: D.O]
Hmm, hmm

[Verse 1: Baekhyun, D.O]
Shimmy-shimmy, ko ko bop, I think I like it
긴장은 down, down, 부끄러 말고
어지러운 마음속에 내가 들어가
익숙한 듯 부드럽게 네게 번져가

[Pre-Chorus: Xiumin, Suho, Kai, Chanyeol]
(Ah-ooh) 고요한 밤이야
(Ah-ooh) 널 위한 밤이야
참을 수가 없어 빠져가, yeah, yeah
너의 몸짓에 난 취해가, yeah, yeah
네가 알던 뻔한 모습의 나를 잊어 오늘
숨겨둔 본능이, shimmy up

[Chorus: Chen, D.O, Suho]
It goes down, down, baby (리듬에 온몸을)
It goes down, down, baby (맡기고)
소리쳐 oh-oh, oh-oh, oh
우린 oh-oh, oh-oh, oh

[Drop: Kai, Chanyeol]
We goin' ko ko bop
(Hey!)

[Verse 2: Xiumin, Sehun]
Shimmy-shimmy, ko ko bop, I think I like it
조금씩 down, down, 수줍어 말고
누가 뭐래도 넌 신경 쓰지 말어
지금 이대로 아름답기만 해
멈춰버렸음 해, baby, are you down?

[Pre-Chorus: Baekhyun, Chen, Suho, Chanyeol]
(Ah-ooh) 마지막 밤이야
(Ah-ooh) 둘만의 밤이야
긴장하지 말고 다가와, yeah, yeah
네 모든 걸 내게 맡겨봐, yeah, yeah
점점 풀려 가는 고삐 더 내려놔 오늘
눈치 보지 말고 shakin' up

[Chorus: Baekhyun, Suho, Chen, Suho & Chen]
It goes down, down, baby (리듬에 온몸을)
It goes down, down, baby (맡기고)
소리쳐 oh-oh, oh-oh, oh
우린 oh-oh, oh-oh, oh

In [6]:
# Since we want to do the sentiment analysis in English, We will try to get the Genius English translation version of the song
song = LyricsGenius.search_song("EXO", "닿은 순간 (Ooh La La La) (English Translation)")

Searching for "EXO" by 닿은 순간 (Ooh La La La) (English Translation)...
Done.


In [7]:
song.lyrics

'[Verse 1: Beakhyun, Kai, Sehun, All]\nWhen I turn my head, my eyes meet yours\nAnd once again, you’re staring at you\nThat smile is a bit too much for me\nFilled with an ease leaving me to burn\nI can’t breathe when I see your eye smile\nIt seems like you know that’s your charm\nYeah, yeah, yeah\n\n[Pre-Chorus: All, Chen, Baekhyun]\nLa la la la\nYour eyes are whispering to me\nLa la la la\nIt’s like they’re asking me to approach\n\n[Chorus: All, Kai, Chen]\nOoh la la la, please allow me\nOoh la la la, into your imagination\nI’ll go right in, so let me in\nI’ll dance in your eyes\nOoh la la la, the moment our gazes meet\n\n[Verse 2: D.O, Suho, Chanyeol]\nRight now, in this moment, the space between us\nIs filled with this strong trembling\nBecoming a rhythm and drawing me in\nLet your body move to it however you’d like\nYeah, she knows\nAn odd smile that doesn’t let on\nSeeing through the curiosity driving me on\nAnd sending a sign\nYeah, yeah, yeah\n\n[Pre-Chorus: All, D.O, Chanyeol, 

#### Get Songs and Lyrics For a Specific Album
Prepare function for data scraping

In [8]:
## RegEx Function To Clean Up Songs
def clean_up(song_title):

    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    return clean_song_title
    
## Function To Scrape Song Titles For Album
def get_all_songs_from_album(artist, album_name):
    
    artist = artist.replace(" ", "-")
    album_name = album_name.replace(" ", "-")
    
    response = requests.get(f"https://genius.com/albums/{artist}/{album_name}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)
        
    return clean_songs

In [9]:
# Function To Download Lyrics For All Songs in Album including English translations
def download_album_lyrics_eng(artist, album_name, client_access_token, include_translations=False):

    #Set up LyricsGenius with your Genius API client access token
    #client_access_token = Your-Client-Access-Token
    
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    
    # With the function that we previously created, go to Genius.com and get all song titles for a particular artist's album
    clean_songs = get_all_songs_from_album(artist, album_name)
    
    for song in clean_songs:
        if include_translations:
            song_search_title = f"{song} (English Translation)" #to get the Englisg Translation 
        else:
            song_search_title = song
        
        song_object = LyricsGenius.search_song(song_search_title, artist)
        
        if song_object:
            #Do some cleaning and prep for the filename of the song
            artist_title = artist.replace(" ", "-")
            album_title = album_name.replace(" ", "-")
            song_title = song.replace("/", "-").replace(" ", "-")
            
            if include_translations:
                directory_name = f"{artist_title}_{album_title}"
            else:
                directory_name = f"{artist_title}_{album_title}"
            
            Path(directory_name).mkdir(parents=True, exist_ok=True)
            
            #Establish the filename for each song inside a directory that begins with the artist's name and album title
            custom_filename = f"{directory_name}/{song_title}"
            
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)
        else:
            print(f'No lyrics found for {song_search_title}')

In [10]:
# Specify Your Artists + Albums of Interest
download_album_lyrics_eng("EXO", "EXIST", client_access_token, include_translations=True)
download_album_lyrics_eng("EXO", "LOVE SHOT", client_access_token, include_translations=True)
download_album_lyrics_eng("EXO", "The War", client_access_token, include_translations=True)

Searching for "Cream Soda (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Cream-Soda.txt.
Searching for "Regret It (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Regret-It.txt.
Searching for "Hear Me Out (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Hear-Me-Out.txt.
Searching for "Private Party (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Private-Party.txt.
Searching for "Cinderella (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Cinderella.txt.
Searching for "No Makeup (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/No-Makeup.txt.
Searching for "Love Fool (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Love-Fool.txt.
Searching for "Another Day (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Another-Day.txt.
Searching for "Let Me In (English Translation)" by EXO...
Done.
Wrote EXO_EXIST/Let-Me-In.txt.
Searching for "Love Shot (English Translation)" by EXO...
Done.
Wrote EXO_LOVE-SHOT/Love-Shot.txt.
Searching for "Tempo (Engl

In [11]:
# from pathlib import Path ; need
# Specify the folder names with the lyric data from Genius
directory_paths = ['EXO_EXIST/',
                   'EXO_LOVE-SHOT/',
                   'EXO_The-War/',]

In [12]:
import re
import string

def clean_text(text):
    
    # Some light data cleaning - you will need to adjust based on your data
    text = re.sub('\d*Embed', ' ', text) # remove ending text with number + Embed
    
    return text

In [13]:
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', 0)

EXO_lyrics = pd.DataFrame({"Album": [],
                          "Title": [],
                          "Lyrics": []})

idx = 0

for i, album in enumerate(directory_paths):
    
    album_name = album[4:-1].replace("-", " ")
        
    for song in Path(album).glob('*.txt'):
        
        song_name = str(song).replace("-", " ").split("/")[1][:-4]
                
        full_text = open(song, encoding="utf-8")
        lyrics_list = full_text.readlines()[1:] #read()
        lyrics_list = [l.replace("\n", "") for l in lyrics_list]
        lyrics = ' '.join(lyrics_list)
        lyrics = clean_text(lyrics)
        full_text.close()
        
        EXO_lyrics.loc[idx] = [album_name, song_name, lyrics]
        idx += 1

In [14]:
EXO_lyrics

,Album,Title,Lyrics
0,EXIST,Cinderella,"City view that has become more colorful beyond our eyes A door that urges you like a lie A dance that was cut off in an instant, oh-woah-woah The clock is already twelve (Late) Unbelievably Before the rush of time (No way) No one can stop you In the waking darkness Ooh, na-na-na you like this I don't want to let you go, Cinderella La-la-la-la-la-la, Cinderella La-la-la-la, Cinderella La-la-la-la-la-la, Cinderella La-la-la-la, Cinderella Ayy, turn everything upside down like a fairy tale, uh My heart is already pounding Ayy, it's just you ain't nobody else Break all the rules that confined you (Hold up) You will disappear and I will stay here I don't want a glass shoes Please don't leave me like this (No) Don't break our dream In this familiar scene (Wait) I lost you in the end So that I won't regret all night (Just wait) I have to change this ending In the waking darkness Ooh, na-na-na you like this I don't want to let you go Cinderella La-la-la-la-la-la, Cinderella La-la-la-la, Cinderella La-la-la-la-la-la, Cinderella La-la-la-la, Cinderella Can't you stay by my side Until the sun rises tomorrow, baby Stay like this forever Cinderella Woah, Cinderella La-la-la-la-la-la, Cinderella La-la-la-la, Cinderella La-la-la-la-la-la, Cinderella La-la-la-la, Cinderella"
1,EXIST,Cream Soda,"Each swirl deepens the flavor, babe Feeling parched and a little bit reckless Won't settle for anything less A delightful mouthful Baby, come closer, a delightful mouthful Never felt this kind of attraction before This tension, like a bottle of cola shaken You're a sparkling drink, pop, already opened Can't think straight no more In front of love, I'm tipsy, baby Pretending to be picky, even if I measure it here and there I just know this tastes too good to be healthy Uncontrollable appetite A taste that shook my everything I need all ya cream soda The more it melts, the sweeter it gets Take my heart out I need all ya cream soda Feel that tinglin' (Cream) That silky smooth cream (Soda) Each swirl deepens the flavor, baby (Cream) Baby go dum dum, shakin' my thinkin' Just a sip, but already brain freezin' Give me that yum, yum, tasty and teasin' Got me wrapped around your little flavor Yeah, sweet like a bon-bon dessert With one sip, instantly dumb, dumb befooled Pumped the brakes, but instincts are so fast Can't stop here, you know I gon' get it, get it Uncontrollable appetite (Get it, get it) A taste that shook my everything I need all ya cream soda The more it melts, the sweeter it gets Take my heart out I need all ya cream soda Foam overflowing, fill it up (Oh, fill it up) Top it off gently, get it up (Oh, get it up) Not one drop lost, drink it all It's spreading love so sweet Wanna dive to their depths Your crystal-clear waters so cool it hurts Keep me wanting Thaw your heart just a bit Can I get just a taste of it? Appetite haunting my heart, got me hooked I need all ya cream soda (Oh yeah, soda) The slower the sip, the sweeter How does it end, I wanna know I need all ya cream soda So baby, tonight whet my appetite cream soda Feel that tinglin', that silky smooth cream Each swirl deepens the flavor, baby Feeling parched and a little bit reckless Won't settle for anything less"
2,EXIST,Hear Me Out,"Look babe, I like you even more I spread my wings and soar on your love, let's fly together I know I mess up, baby, I can't help it Trying to push me away, just save it for later 'Cause I've been thinking bout some time with you All this is just the beginning for me Want absolutely nothing in return, my baby Gazing at the same skies all night long, my favorite thing Girl, just Don't think too hard We are only wasting time, love Hear me out, hear me out, hear me out So what? It's all good Hear me out, hear mе out, hear me out No need to hesitatе Oh, babe, hold me closely Thinking too much messes with your flow You know I don't play games Never have you shown me Lookin' so good, busy getting ready to leave Treatin' me

In [15]:
# Recheck
print(EXO_lyrics)

        Album                        Title  \
0   EXIST      Cinderella                    
1   EXIST      Cream Soda                    
2   EXIST      Hear Me Out                   
3   EXIST      Regret It                     
4   EXIST      Another Day                   
5   EXIST      Love Fool                     
6   EXIST      Private Party                 
7   EXIST      Let Me In                     
8   EXIST      No Makeup                     
9   LOVE SHOT  Tempo                         
10  LOVE SHOT  Sign                          
11  LOVE SHOT  트라우마 (Trauma)                 
12  LOVE SHOT  가끔 (With You)                 
13  LOVE SHOT  Gravity                       
14  LOVE SHOT  닿은 순간 (Ooh La La La)          
15  LOVE SHOT  Love Shot                     
16  LOVE SHOT  24 7                          
17  LOVE SHOT  여기 있을게 (Smile On My Face)     
18  LOVE SHOT  오아시스 (Oasis)                  
19  LOVE SHOT  Wait                          
20  LOVE SHOT  Damage             

Due to EXO's songs having both Chinese and Korean versions, some lyrics are duplicated. Therefore, we will remove these duplicate entries from the dataset.

In [16]:
# Drop rows where 'Title' contains "(Chinese Ver.)"
EXO_lyrics = EXO_lyrics[~EXO_lyrics['Title'].str.contains(r"\(Chinese Ver.\)")]

# Print the updated DataFrame
print(EXO_lyrics)

        Album                        Title  \
0   EXIST      Cinderella                    
1   EXIST      Cream Soda                    
2   EXIST      Hear Me Out                   
3   EXIST      Regret It                     
4   EXIST      Another Day                   
5   EXIST      Love Fool                     
6   EXIST      Private Party                 
7   EXIST      Let Me In                     
8   EXIST      No Makeup                     
9   LOVE SHOT  Tempo                         
10  LOVE SHOT  Sign                          
11  LOVE SHOT  트라우마 (Trauma)                 
12  LOVE SHOT  가끔 (With You)                 
13  LOVE SHOT  Gravity                       
14  LOVE SHOT  닿은 순간 (Ooh La La La)          
15  LOVE SHOT  Love Shot                     
16  LOVE SHOT  24 7                          
17  LOVE SHOT  여기 있을게 (Smile On My Face)     
18  LOVE SHOT  오아시스 (Oasis)                  
19  LOVE SHOT  Wait                          
20  LOVE SHOT  Damage             

#### Export the Lyrics as a CSV File

In [17]:
EXO_lyrics.to_csv("EXO_genius_data.csv", index=False)

Reference and Resort
-  Melanie Walsh's Intro to Cultural Analytics & Python book:
https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/08-Collect-Genius-Lyrics.html
- Arthur's Tutorial "Music and Lyrics : How to get the lyrics of your favorite songs and artists from Genius API ?"
https://medium.com/@pacearthur/music-and-lyrics-how-to-get-the-lyrics-of-your-favorite-songs-and-artists-from-genius-api-966b0d86b10a